In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df=pd.read_csv("~/Downloads/ECTA 2023 Dataset.csv") 
features = ['Relative Humidity', 'Wind Speed', 'Visibility', 'Pressure', 'Wind_Chill', 'Dewpoint Temp'] 
target = 'Temperature'

df.dropna() 
X=df[features].values 
Y=df[target].values

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2, random_state=40) 
scaler=StandardScaler() #Standardises the data 
X_train=scaler.fit_transform(X_train) #Fits and then transforms. 
X_test=scaler.transform(X_test) #Leaking data is bad, so mu and sigma are stored from the fit transform we did earlier as attributes of the scaler object and reused. 

def build_ann(lr,neurons1,neurons2): 
    model=keras.Sequential([
        keras.layers.Input(shape=(X_train.shape[1],)),
        keras.layers.Dense(neurons1,activation='relu'),
        keras.layers.Dense(neurons2,activation='relu'),
        keras.layers.Dense(1,activation='linear')
    ])
    optimizer=keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer,loss='mean_squared_error',metrics=['mae'])
    return model
def evaluate(params): #Simulating a forward pass
    lr,n1,n2,bs=params
    n1,n2,bs=int(n1),int(n2),int(bs)
    model=build_ann(lr,n1,n2)
    model.fit(X_train,Y_train,epochs=10,batch_size=bs,verbose=1)
    loss,mae=model.evaluate(X_test,Y_test,verbose=1)
    return mae

num_particle=8 #Number of particles
num_iterations=5 #Number of iterations (not epochs!)
inertia=0.5 
c1,c2=2.0,2.0 #c1 is cognitive coefficient while c2 is social coefficient. c1 dictates how much it listens to itself, c2 how much it listens to others.
lower_bounds=[0.001,10,10,16]  
upper_bounds=[0.1,100,100,128]

particles=np.random.uniform(lower_bounds,upper_bounds,(num_particle,4)) #4 is because 4 params, lr n1 n2 and bs
velocities=np.random.uniform(-1,1,(num_particle,4))

pbest=particles.copy() #Make a copy of the best current set
pbest_score=np.array([evaluate(p) for p in particles]) #Initially each particle thinks it's the best
gbest=pbest[np.argmin(pbest_score)] #Global best position
gbest_score=np.min(pbest_score) #Global best score
print(f"Initial Global Best MAE: {gbest_score:.4f}")


#Main PSO Loop
for _ in range(num_iterations): 
    for i in range(num_particle): #For each particle
        r1,r2=np.random.rand(4), np.random.rand(4) #Random influence vectors, 1 for itself and 2 for global. helps avoid particle getting stuck in local minima
        velocities[i]=(inertia*velocities[i] + c1*r1*(pbest[i]-particles[i])+ c2*r2*(gbest-particles[i])) #Update velocities
        particles[i]+=velocities[i] #Update position
        particles[i]=np.clip(particles[i],lower_bounds,upper_bounds) #Keeps the values within our lower and upper bounds. makes sure the particles don't fly off beyond what we want to consider
        score=evaluate(particles[i]) 
        if score<pbest_score[i]: #Update if we find a better score
            pbest[i]=particles[i]
            pbest_score[i]=score 
            print(f"Particle {i+1}: MAE = {score:.4f}") 
            if score<gbest_score:
                gbest=particles[i] 
                gbest_score=score
                print(f"New Global Best MAE found: {gbest_score:.4f}")

best_lr,best_n1,best_n2,best_bs=gbest #Get the best values of the parameters
mae=evaluate(gbest) #One final forward pass through the ANN using the hyperparameters
print(f"Best Params: lr={best_lr:.5f}, n1={int(best_n1)}, n2={int(best_n2)}, batch={int(best_bs)}") 
print(f"Final MAE: {mae:.4f}") 

2025-04-18 01:24:14.073727: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
2572/2572 ━━━━━━━━━━━━━━━━━━━━ 2s 556us/step - loss: 2.8254 - mae: 0.6105
Epoch 2/10
2572/2572 ━━━━━━━━━━━━━━━━━━━━ 2s 590us/step - loss: 0.1507 - mae: 0.2509
Epoch 3/10
2572/2572 ━━━━━━━━━━━━━━━━━━━━ 2s 606us/step - loss: 0.1020 - mae: 0.2258
Epoch 4/10
2572/2572 ━━━━━━━━━━━━━━━━━━━━ 1s 562us/step - loss: 0.0748 - mae: 0.1856
Epoch 5/10
2572/2572 ━━━━━━━━━━━━━━━━━━━━ 1s 516us/step - loss: 0.0568 - mae: 0.1627
Epoch 6/10
2572/2572 ━━━━━━━━━━━━━━━━━━━━ 1s 529us/step - loss: 0.0632 - mae: 0.1748
Epoch 7/10
2572/2572 ━━━━━━━━━━━━━━━━━━━━ 2s 604us/step - loss: 0.0591 - mae: 0.1729
Epoch 8/10
2572/2572 ━━━━━━━━━━━━━━━━━━━━ 2s 583us/step - loss: 0.0640 - mae: 0.1795
Epoch 9/10
2572/2572 ━━━━━━━━━━━━━━━━━━━━ 2s 584us/step - loss: 0.0663 - mae: 0.1826
Epoch 10/10
2572/2572 ━━━━━━━━━━━━━━━━━━━━ 2s 577us/step - loss: 0.0586 - mae: 0.1665
603/603 ━━━━━━━━━━━━━━━━━━━━ 0s 477us/step - loss: 0.0251 - mae: 0.1265
Epoch 1/10
660/660 ━━━━━━━━━━━━━━━━━━━━ 1s 665us/step - loss: 5.6337 - mae: 0